In [1]:
from ZW_model import PSI
from ZW_utils import std_classes,dataloading
from ZW_dataset import PSI_Dataset

classes = std_classes
data_split_ratio = 0.85
batch_size = 100
max_epochs = 100
learning_rate = 1e-3
block_size = 22
n_embd = 32  # 32
n_head = 4  # 4
n_layer = 2  # 2
dropout = 0.1  # 0.1
vocab_size = len(classes)
model = PSI(vocab_size, n_embd, n_head, n_layer, block_size, dropout)

In [6]:
import numpy as np
from config import DATA_DIRECTORY
from torch.utils.data import DataLoader
import torch

# layouts = np.load(DATA_DIRECTORY/"v22DF_m2_sorted_layouts.npy", allow_pickle=True)
# results = np.load(DATA_DIRECTORY/"v22DF_m2_sorted_results.npy", allow_pickle=True)
layouts = np.load("GPT_NA/initial_10k.npy", allow_pickle=True)
results = np.load("GPT_NA/results_initial_10k.npy", allow_pickle=True)
l2 = []
r2 = []
cutoff = 143.957
for i,r in enumerate(results):
    if r > 0:
        l2.append(layouts[i])
        r2.append(r)
layouts = np.asanyarray(l2)
results = np.asanyarray(r2)
from split_functions import string_to_equipment,enforced_uniqueness
# equipments = string_to_equipment(layouts)
# layouts,_ = enforced_uniqueness(equipments)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
data = PSI_Dataset(layouts, results, classes,block_size,training_type="standard")
training_set, validation_set = torch.utils.data.random_split(data, [int(data_split_ratio*len(data)), len(data)-int(data_split_ratio*len(data))])
train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=True)

for epoch in range(max_epochs):
    model.train()
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        y_pred = model(x)
        y_pred = y_pred.view(y.size(0),block_size)
        y = y.to(torch.float32)
        loss = torch.nn.functional.mse_loss(y_pred, y)
        loss.backward()
        optimizer.step()
    val_loss = 0
    with torch.no_grad():
        model.eval()
        for i, (x, y) in enumerate(val_loader):
            y_pred = model(x)
            y_pred = y_pred.view(y.size(0),block_size)
            y = y.to(torch.float32)
            loss = torch.nn.functional.mse_loss(y_pred, y)
            val_loss += loss.item()
        val_loss /= len(val_loader)
    print(f"Epoch {epoch} - Validation loss: {val_loss}")

Designs in the dataset: 1552
Input shape: torch.Size([1552, 22]) Output shape: torch.Size([1552, 22])
Epoch 0 - Validation loss: 36469.700520833336
Epoch 1 - Validation loss: 32430.129231770832
Epoch 2 - Validation loss: 43360.381510416664
Epoch 3 - Validation loss: 36184.130208333336
Epoch 4 - Validation loss: 31649.482747395832
Epoch 5 - Validation loss: 42193.388671875
Epoch 6 - Validation loss: 31411.5224609375
Epoch 7 - Validation loss: 30176.516927083332
Epoch 8 - Validation loss: 38394.544270833336
Epoch 9 - Validation loss: 30967.652669270832
Epoch 10 - Validation loss: 41733.4140625
Epoch 11 - Validation loss: 34869.491536458336
Epoch 12 - Validation loss: 31505.180338541668
Epoch 13 - Validation loss: 31485.048828125
Epoch 14 - Validation loss: 28239.730794270832
Epoch 15 - Validation loss: 39478.657552083336
Epoch 16 - Validation loss: 30577.629557291668
Epoch 17 - Validation loss: 30003.701822916668
Epoch 18 - Validation loss: 34838.7109375
Epoch 19 - Validation loss: 35316

In [7]:
model.eval()
model(torch.tensor([[0,1],[0,2],[0,3],[0,4],[0,5],[0,7],[0,9],[0,11]]))
# model(torch.tensor([[0,5,1],[0,5,2],[0,5,3],[0,5,4],[0,5,5],[0,5,7],[0,5,9],[0,5,11]]))
# model(torch.tensor([[0,5,4,1],[0,5,4,2],[0,5,4,3],[0,5,4,4],[0,5,4,5],[0,5,4,7],[0,5,4,9],[0,5,4,11]]))

tensor([[[255.2910],
         [213.3581]],

        [[255.2910],
         [255.8791]],

        [[255.2910],
         [211.1716]],

        [[255.2910],
         [255.9067]],

        [[255.2910],
         [255.8386]],

        [[255.2910],
         [255.8868]],

        [[255.2910],
         [210.0258]],

        [[255.2910],
         [248.1066]]], grad_fn=<ViewBackward0>)